# Загружаем необходимые библиотеки

In [1]:
!pip install gdown
!apt-get install unrar
!pip install transformers==4.15
!pip install SentencePiece
!pip install pandas
!pip install datasets
!pip install jiwer jiwer

  Using cached gdown-4.4.0-py3-none-any.whl
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
  Using cached transformers-4.15.0-py3-none-any.whl (3.4 MB)
  Using cached huggingface_hub-0.4.0-py3-none-any.whl (67 kB)
  Using cached sacremoses-0.0.47-py2.py3-none-any.whl (895 kB)
  Using cached tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Using cached datasets-1.18.3-py3-none-any.whl (311 kB)
  Using cached multiprocess-0.70.12.2-py37-none-any.whl (112 kB)
  Using cached xxhash-3.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (212 kB)
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)


  Using cached jiwer-2.3.0-py3-none-any.whl (15 kB)
  Using cached python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl


In [2]:
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu101/torch-1.7.1%2Bcu101-cp37-cp37m-linux_x86_64.whl (735.4 MB)
  Using cached https://download.pytorch.org/whl/cu101/torchvision-0.8.2%2Bcu101-cp37-cp37m-linux_x86_64.whl (12.8 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.7.0+cu101
    Uninstalling torchvision-0.7.0+cu101:
      Successfully uninstalled torchvision-0.7.0+cu101


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json

# Делим датасет на трейн и вал

In [4]:
train_csv = pd.read_csv('data/train_recognition/labels.csv')
train_csv = train_csv.sample(frac = 1)

train_data = dict(train_csv[['file_name','text']].values)

In [5]:

train_data = [(k, v) for k, v in train_data.items()]
print('train len', len(train_data))

split_coef = 0.75
train_len = int(len(train_data)*split_coef)

train_data_splitted = train_data[:train_len]
val_data_splitted = train_data[train_len:]

print('train len after split', len(train_data_splitted))
print('val len after split', len(val_data_splitted))


"\ntrain_data = [(k, v) for k, v in train_data.items()]\nprint('train len', len(train_data))\n\nsplit_coef = 0.75\ntrain_len = int(len(train_data)*split_coef)\n\ntrain_data_splitted = train_data[:train_len]\nval_data_splitted = train_data[train_len:]\n\nprint('train len after split', len(train_data_splitted))\nprint('val len after split', len(val_data_splitted))\n"

In [6]:
with open('data/train_recognition/train_labels_splitted.json', 'w') as f:
    json.dump(dict(train_data_splitted), f)
    
with open('data/train_recognition/val_labels_splitted.json', 'w') as f:
    json.dump(dict(val_data_splitted), f)

# Загружаем датасет

In [7]:
with open("data/train_recognition/train_labels_splitted.json") as train_file:
    dict_train = json.load(train_file)

with open('data/train_recognition/val_labels_splitted.json') as train_file:
    dict_test = json.load(train_file)

# converting json dataset from dictionary to dataframe
train_df= pd.DataFrame.from_dict(dict_train, orient='index', columns=['text'])
train_df.index.rename('file_name', inplace=True)
train_df.reset_index(level=0, inplace=True)
train_df.head()

test_df= pd.DataFrame.from_dict(dict_test, orient='index', columns=['text'])
test_df.index.rename('file_name', inplace=True)
test_df.reset_index(level=0, inplace=True)
test_df.head()

,file_name,text
0,7061_eng.png,women
1,15346.png,ед.ч.
2,41393.png,"свой,"
3,59147.png,нашей
4,118365.png,ться


In [8]:
import torch
import math
import torchvision
import cv2
import random
import numpy as np


class RescalePaddingImage:
    def __init__(self, output_height, output_width):
        self.output_height = output_height
        self.output_width = output_width

    def __call__(self, image):
        h, w = image.shape[:2]
        # width proportional to change in  height
        new_width = int(w*(self.output_height/h))
        # new_width cannot be bigger than output_width
        new_width = min(new_width, self.output_width)
        image = cv2.resize(image, (new_width, self.output_height),
                           interpolation=cv2.INTER_LINEAR)
        if new_width < self.output_width:
            image = np.pad(
                image, ((0, 0), (0, self.output_width - new_width), (0, 0)),
                'constant', constant_values=0)
        return image


class Normalize:
    def __call__(self, img):
        img = img.astype(np.float32) / 255
        return img


class ToTensor:
    def __call__(self, arr):
        arr = torch.from_numpy(arr)
        return arr


class MoveChannels:
    """Move the channel axis to the zero position as required in pytorch."""

    def __init__(self, to_channels_first=True):
        self.to_channels_first = to_channels_first

    def __call__(self, image):
        if self.to_channels_first:
            return np.moveaxis(image, -1, 0)
        else:
            return np.moveaxis(image, 0, -1)


class UseWithProb:
    def __init__(self, transform, prob=0.5):
        self.transform = transform
        self.prob = prob

    def __call__(self, image):
        if random.random() < self.prob:
            image = self.transform(image)
        return image


class RandomGaussianBlur:
    """Apply Gaussian blur with random kernel size
    Args:
        max_ksize (int): maximal size of a kernel to apply, should be odd
        sigma_x (int): Standard deviation
    """

    def __init__(self, max_ksize=5, sigma_x=20):
        assert max_ksize % 2 == 1, "max_ksize should be odd"
        self.max_ksize = max_ksize // 2 + 1
        self.sigma_x = sigma_x

    def __call__(self, image):
        kernal_size = tuple(2 * np.random.randint(0, self.max_ksize, 2) + 1)
        blured_image = cv2.GaussianBlur(image, kernal_size, self.sigma_x)
        return blured_image


def img_crop(img, bbox):
    return img[bbox[1]:bbox[3], bbox[0]:bbox[2]]


def random_crop(img, size):
    tw = size[0]
    th = size[1]
    h, w = img.shape[:2]
    if ((w - tw) > 0) and ((h - th) > 0):
        x1 = random.randint(0, w - tw)
        y1 = random.randint(0, h - th)
    else:
        x1 = 0
        y1 = 0
    img_return = img_crop(img, (x1, y1, x1 + tw, y1 + th))
    return img_return, x1, y1


class RandomCrop:
    def __init__(self, rnd_crop_min, rnd_crop_max=1):
        self.factor_max = rnd_crop_max
        self.factor_min = rnd_crop_min

    def __call__(self, img):
        factor = random.uniform(self.factor_min, self.factor_max)
        size = (
            int(img.shape[1]*factor),
            int(img.shape[0]*factor)
        )
        img, x1, y1 = random_crop(img, size)
        return img


def largest_rotated_rect(w, h, angle):
    """
    https://stackoverflow.com/a/16770343
    Given a rectangle of size wxh that has been rotated by 'angle' (in
    radians), computes the width and height of the largest possible
    axis-aligned rectangle within the rotated rectangle.
    Original JS code by 'Andri' and Magnus Hoff from Stack Overflow
    Converted to Python by Aaron Snoswell
    """

    quadrant = int(math.floor(angle / (math.pi / 2))) & 3
    sign_alpha = angle if ((quadrant & 1) == 0) else math.pi - angle
    alpha = (sign_alpha % math.pi + math.pi) % math.pi

    bb_w = w * math.cos(alpha) + h * math.sin(alpha)
    bb_h = w * math.sin(alpha) + h * math.cos(alpha)

    gamma = math.atan2(bb_w, bb_w) if (w < h) else math.atan2(bb_w, bb_w)

    delta = math.pi - alpha - gamma

    length = h if (w < h) else w

    d = length * math.cos(alpha)
    a = d * math.sin(alpha) / math.sin(delta)

    y = a * math.cos(gamma)
    x = y * math.tan(gamma)

    return (
        bb_w - 2 * x,
        bb_h - 2 * y
    )


def crop_around_center(image, width, height):
    """
    https://stackoverflow.com/a/16770343
    Given a NumPy / OpenCV 2 image, crops it to the given width and height,
    around it's centre point
    """

    image_size = (image.shape[1], image.shape[0])
    image_center = (int(image_size[0] * 0.5), int(image_size[1] * 0.5))

    if(width > image_size[0]):
        width = image_size[0]

    if(height > image_size[1]):
        height = image_size[1]

    x1 = int(image_center[0] - width * 0.5)
    x2 = int(image_center[0] + width * 0.5)
    y1 = int(image_center[1] - height * 0.5)
    y2 = int(image_center[1] + height * 0.5)

    return image[y1:y2, x1:x2]


class RandomRotate:
    """Random image rotate around the image center
    Args:
        max_ang (float): Max angle of rotation in deg
    """

    def __init__(self, max_ang=0):
        self.max_ang = max_ang

    def __call__(self, img):
        h, w, _ = img.shape

        ang = np.random.uniform(-self.max_ang, self.max_ang)
        M = cv2.getRotationMatrix2D((w/2, h/2), ang, 1)
        img = cv2.warpAffine(img, M, (w, h))

        w_cropped, h_cropped = largest_rotated_rect(w, h, math.radians(ang))
        img = crop_around_center(img, w_cropped, h_cropped)
        return img


class ImageResize:
    def __init__(self, height, width):
        self.height = height
        self.width = width

    def __call__(self, image):
        image = cv2.resize(image, (self.width, self.height),
                           interpolation=cv2.INTER_LINEAR)
        return image


def get_train_transforms(height, width, prob=0.2):
    transforms = torchvision.transforms.Compose([
        UseWithProb(RandomGaussianBlur(max_ksize=7), prob=prob),
        UseWithProb(RandomRotate(5), prob),
        # ImageResize(height, width),
        # UseWithProb(RandomCrop(rnd_crop_min=0.85), prob),
        # RescalePaddingImage(height, width),
        # MoveChannels(to_channels_first=True),
        # Normalize(),
        # ToTensor()
    ])
    return transforms


def get_val_transforms(height, width):
    transforms = torchvision.transforms.Compose([
        RescalePaddingImage(height, width),
        # ImageResize(height, width),
        MoveChannels(to_channels_first=True),
        Normalize(),
        ToTensor()
    ])
    return transforms

'""\ndef get_val_transforms(height, width):\n    transforms = torchvision.transforms.Compose([\n        RescalePaddingImage(height, width),\n        # ImageResize(height, width),\n        MoveChannels(to_channels_first=True),\n        Normalize(),\n        ToTensor()\n    ])\n    return transforms\n'

In [9]:
import torch
from torch.utils.data import Dataset
from PIL import Image

class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128, transforms=None):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text 
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        
        # perform augmentations
        if self.transforms:
            opencv_image = self.transforms(cv2.imread(self.root_dir + file_name))
            image = Image.fromarray(opencv_image)
        else:
            image = Image.open(self.root_dir + file_name).convert("RGB")
            
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, padding="max_length", 
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [10]:
from transformers import AutoTokenizer, XLMRobertaTokenizerFast, DeiTFeatureExtractor
from contextlib import contextmanager


class CustomTrOCRProcessor:

    def __init__(self, feature_extractor, tokenizer):
        self.feature_extractor = feature_extractor
        self.tokenizer = tokenizer
        self.current_processor = self.feature_extractor

    def save_pretrained(self, save_directory):
        self.feature_extractor.save_pretrained(save_directory)
        self.tokenizer.save_pretrained(save_directory)


    def __call__(self, *args, **kwargs):
        return self.current_processor(*args, **kwargs)

    def batch_decode(self, *args, **kwargs):
        return self.tokenizer.batch_decode(*args, **kwargs)

    def decode(self, *args, **kwargs):
        return self.tokenizer.decode(*args, **kwargs)
      

    @contextmanager
    def as_target_processor(self):
        self.current_processor = self.tokenizer
        yield
        self.current_processor = self.feature_extractor

In [11]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoFeatureExtractor, XLMRobertaTokenizer
import pickle
feature_extractor = AutoFeatureExtractor.from_pretrained('microsoft/trocr-small-handwritten')
tokenizer = XLMRobertaTokenizer.from_pretrained('microsoft/trocr-small-handwritten')

processor = CustomTrOCRProcessor(feature_extractor, tokenizer)
#filehandler = open('./drive/MyDrive/Распознавание_рукописного_текста/processor1.pkl','wb')
#pickle.dump(processor, filehandler)

In [12]:
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-small-handwritten")
# torch.save(model, './drive/MyDrive/Распознавание_рукописного_текста/model1.pth')

model.to(device)

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-handwritten and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


VisionEncoderDecoderModel(
  (encoder): DeiTModel(
    (embeddings): DeiTEmbeddings(
      (patch_embeddings): PatchEmbeddings(
        (projection): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DeiTEncoder(
      (layer): ModuleList(
        (0): DeiTLayer(
          (attention): DeiTAttention(
            (attention): DeiTSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): DeiTSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): DeiTIntermediate(
            (dense): Linear(in_features=

In [13]:
from torch.utils.data import DataLoader

#processor = CustomTrOCRProcessor(feature_extractor, tokenizer)
#processor = pickle.load(open('./drive/MyDrive/Распознавание_рукописного_текста/processor.pkl', 'rb'))
PATH = './data/train_recognition/images/'
train_dataset = IAMDataset(root_dir=PATH,
                           df=train_df,
                           transforms = get_train_transforms(),
                           processor=processor)
eval_dataset = IAMDataset(root_dir=PATH,
                           df=test_df,
                           processor=processor)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True, num_workers=4)
eval_dataloader = DataLoader(eval_dataset, batch_size=64, num_workers=4)

In [14]:
def compute_cer(pred_ids, label_ids):
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return cer

In [15]:
from datasets import load_metric
from transformers import AdamW
from tqdm.notebook import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)
scaler = torch.cuda.amp.GradScaler()
cer_metric = load_metric("cer")

2022-03-02 11:56:34.324038: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [16]:
""""
after_training=False
if after_training:
    opt_model_dict = torch.load('../input/check8/check0CER-0.09763638173246426.pt')
    optimizer.load_state_dict(opt_model_dict['optimizer'])
    model.load_state_dict(opt_model_dict['torch'])
model
"""

'"\nafter_training=False\nif after_training:\n    opt_model_dict = torch.load(\'../input/check8/check0CER-0.09763638173246426.pt\')\n    optimizer.load_state_dict(opt_model_dict[\'optimizer\'])\n    model.load_state_dict(opt_model_dict[\'torch\'])\nmodel\n'

In [17]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [18]:
for epoch in range(20):
    model.train()
    train_loss = 0.0
    for batch in tqdm(train_dataloader):
        # get the inputs
        for k,v in batch.items():
            batch[k] = v.to(device)

        # forward + backward + optimize
        with torch.cuda.amp.autocast():
            outputs = model(**batch)
            loss = outputs.loss
  
        optimizer.zero_grad()  
        scaler.scale(loss).backward()
        scaler.step(optimizer)  
        scaler.update()

        train_loss += loss.item()

    # scheduler.step()
    print(f"Loss after epoch {epoch}:", train_loss/len(train_dataloader))

    # evaluate
    model.eval()
    valid_cer = 0.0
    with torch.no_grad():
        for batch in tqdm(eval_dataloader):
            # run batch generation
            
            outputs = model.generate(batch["pixel_values"].to(device))
            # compute metrics
            try:
                cer = compute_cer(pred_ids=outputs, label_ids=batch["labels"])
            except Exception as e:
                pass
            valid_cer += cer

    print("Validation CER:", valid_cer / len(eval_dataloader))
    torch.save({'torch': model.state_dict(),
                'optimizer': optimizer.state_dict()}, 
               f"check{epoch}CER-{valid_cer / len(eval_dataloader)}.pt") 

model.save_pretrained(".")

  0%|          | 0/3790 [00:00<?, ?it/s]

Loss after epoch 0: 1.2046340216511473


  0%|          | 0/632 [00:00<?, ?it/s]

Validation CER: 0.17721372074032143


  0%|          | 0/3790 [00:00<?, ?it/s]

...

Loss after epoch 14: 0.07337625361168093


  0%|          | 0/632 [00:00<?, ?it/s]

Validation CER: 0.07991520817471433


  0%|          | 0/3790 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
!pip install GPUtil
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()      